# python3.5の仮想環境を構築してpygameを試す
pygameは、現時点でPython 3.5の環境じゃないとインストールが出来ないので、仮想環境を作ってpygameを試す。
  - pygameが動くpython 3.5の仮想環境を構築する。 `>conda create -n py35 python=3.5`
  - pygameをインストールする。 `>conda install -c cogsci pygame`
  - プログラムを試す。

## 1. python3.5の仮想環境構築 (Anaconda Promptで実行)
### 1-1. 現在の環境を確認
baseの環境のみが存在している。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
~~~

### 1-2. py35という名前で仮想環境を作成
現時点でpygameがpython3.5に対応しているため、python3.5をインストールする。
~~~
(base) C:\***>conda create -n py35 python=3.5

~~~

### 1-3. 仮想環境が出来たか確認
py35という名前の仮想環境が出来ている。\*が付いた方がactiveな環境。
~~~
(base) C:\***>conda info -e
# conda environments:
base                  *  C:\***\anaconda3
py35                     C:\***\anaconda3\envs\py35
~~~

### 1-4. 仮想環境を切り替え
py35をactiveにする。
~~~
(base) C:\***>activate py35
base                     C:\***\anaconda3
py35                  *  C:\***\anaconda3\envs\py35
~~~

### 1-5. py35環境にJupyterとSpyderをインストールする
~~~
(py35) C:\***>conda install jupyter
(py35) C:\***>conda install spyder
~~~

### 1-6. 環境をアップデートしないとうまく動かなかったので、アップデートする。

~~~
>conda update –-all
~~~

### py35の仮想環境を削除する場合。
一旦対象をdeactivate。（baseをacctivate）し、ターミナルを再起動する。その後、py35の削除処理を行う。
~~~
(py35) C:\***>conda.bat activate base
(base) C:\***>conda remove -n py35 --all
~~~

## 2. pygameのサンプルプログラムを試す
ウィンドウ内をクリックすると赤い粒（Dot）が生成され、ウィンドウ内を有機的に動き回るサンプルプログラム。

In [2]:
import pygame
from pygame.locals import MOUSEBUTTONDOWN, QUIT
import sys
import random
import math

## 2-1. Dotクラス生成

In [ ]:
class Dot:
    
    def __init__(self, x, y, vx, vy):
        self.x, self.y = x, y # position 
        self.vx, self.vy = vx, vy # velocity
        
    def update(self, WINDOW_W, WINDOW_H):
        self.x += self.vx
        self.y += self.vy
        
        #Right edge.
        if self.x > WINDOW_W:
            self.vx, self.x = -self.vx, WINDOW_W
            return
        
        #Left edge.
        if self.x < 0:
            self.vx, self.x = -self.vx, 0
            return
        
        #Upper edge.
        if self.y > WINDOW_H:
            self.vy, self.y = -self.vy, WINDOW_H
            return
        #Lower edge.
        
        if self.y < 0:
            self.vy, self.y = -self.vy, 0
            return

    def draw(self, screen):
        x, y = int(self.x), int(self.y)
        pygame.draw.circle(screen, (255,0,0), (x,y), 5)
    
    # the rule of separation.
    def separation(self, r_s):
        tvx = tvy = c = 0
        
        for a in self.others:
            if a[1] < r_s and a[1] != 0:
                tvx -= (a[0].x - self.x) / a[1]
                tvy -= (a[0].y - self.y) / a[1]
                c += 1
                
        if c != 0:
            self.vx_s, self.vy_s = tvx / c, tvy / c
        else:
            self.vx_s, self.vy_s = self.vx, self.vy
    
    # the rule of alignment.
    def alignment(self, r_a):
        tvx = tvy = c = 0
        
        for a in self.others:
            if a[1] < r_a:
                tvx += a[0].vx
                tvy += a[0].vy
                c += 1
                
        if c != 0:
            self.vx_a, self.vy_a = tvx / c, tvy / c
        else:
            self.vx_a, self.vy_a = self.vx, self.vy
        
        
    # the rule of cohesion.
    def cohesion(self, r_c):
        tx = ty = c = 0
        
        for a in self.others:
            if a[1] < r_c:
                tx += a[0].x
                ty += a[0].y
                c += 1
                
        if c != 0:
            tx, ty = tx / c, ty / c
            d = math.sqrt((tx - self.x)**2 + (ty - self.y)**2)
            if d != 0:
                self.vx_c = (tx - self.x) / d
                self.vy_c = (ty - self.y) / d
        else:
            self.vx_c, self.vy_c = self.vx, self.vy
            
    def rule(self, agent_list, r_s, r_a, r_c):
        self.others = tuple([(a, math.sqrt((a.x - self.x)**2 \
                            + (a.y - self.y)**2)) \
                             for a in agent_list if a != self])
        
        if len(self.others) < 1: return
        
        self.separation(r_s)
        self.alignment(r_a)
        self.cohesion(r_c)
        
        tvx = self.vx_s * 1.0 + self.vx_a * 0.4 + self.vx_c * 0.2
        tvy = self.vy_s * 1.0 + self.vy_a * 0.4 + self.vy_c * 0.2
        
        n = math.sqrt(tvx**2 + tvy**2)
        
        self.vx, self.vy = 2 * tvx / n, 2 * tvy / n
            
            

In [ ]:
#main
WINDOW_W, WINDOW_H = 1000, 700
pygame.init()
#Create window.
screen = pygame.display.set_mode((WINDOW_W, WINDOW_H))
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)
dot_list = []
rule_on = 1

while True:
    clock.tick(60)
    
    # Fill screen with white.
    screen.fill((255, 255, 255))
    
    # Process each agent.
    for d in dot_list:
        if rule_on == 1:
            d.rule(dot_list, 15, 30, 50)
            
        d.update(WINDOW_W, WINDOW_H)
        d.draw(screen)
    
    # Show the number of the agent.
    s1 = "dots : " + str(len(dot_list))
    s2 = "rule : on" if rule_on == 1 else "rule : off"
    
    text = font.render(s1 + "  " + s2, True, (0,0,0))
    screen.blit(text, (1, 1))

    # Process events.
    for e in pygame.event.get():
        # When clicked the left button on mouse.
        if e.type == MOUSEBUTTONDOWN and ( e.button==4 or e.button==5):
            # Set velocity by use a random number. `numpy.random.uniform(low=0.0, high=1.0, size=None)`
            vx = random.uniform(-15, 15)
            vy = random.uniform(-15, 15)
            # Create a dot.
            dot = Dot(e.pos[0], e.pos[1], vx, vy)
            # Add created dot into the dot list.
            dot_list.append(dot)
        
        if e.type == MOUSEBUTTONDOWN and e.button == 1:
            ruel_on = 1 - rule_on
            
        if e.type == QUIT:
            pygame.quit()
            sys.exit()
        
    pygame.display.update()
    